# Experimenting and EDA for Dataset

In [1]:
import pymysql
import pandas as pd
from dotenv import dotenv_values
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from typing import Tuple



In [2]:
# Getting env var
secrets = dotenv_values()

host =secrets['DB_HOST']
name = secrets['DB_NAME']
user = secrets['DB_USER']
pwd = secrets['DB_PWD']


In [3]:
# Connecting to DB

connection = pymysql.connect(
    host = host,
    user = user,
    password = pwd,
    database = name
    )

cursor = connection.cursor()
cursor.execute("SELECT * FROM churn_status")

results = cursor.fetchall()

print(len(results))

cursor.close()
connection.close()

7043


# EDA
Done from local data files

---

To Do List
- Join unscaled and unprocessed datasets. 
- Remove rows without churn label (if any)
- Check for class imbalance -> if have, perform stratified split 
- Split into train-validation (70-30)
- Perform outlier removal only on train set
- After outlier removal, preprocess (categorical encode and scale) columns in both datasets
- Perform PCA to determine which features to use
- Train machine learning model with kfold cross validation

In [3]:
# File paths
ACC_PATH = "data_given/1_account.csv"
ACC_USAGE_PATH = "data_given/2_account_usage.csv"
CHURN_STATUS_PATH = "data_given/3_churn_status.csv"
CITY_PATH = "data_given/4_city.csv"
CUSTOMER_PATH = "data_given/5_customer.csv"

### Joining all unprocessed tables

In [4]:
# Loading DFs
CUSTOMER_DF = pd.read_csv(CUSTOMER_PATH)
ACCOUNT_DF = pd.read_csv(ACC_PATH)
ACC_USAGE_DF = pd.read_csv(ACC_USAGE_PATH)
CHURN_STATUS_DF = pd.read_csv(CHURN_STATUS_PATH)
CITY_DF = pd.read_csv(CITY_PATH)

# Joining customer and account tables on customer_id
DF = pd.merge(CUSTOMER_DF, ACCOUNT_DF, on='customer_id', how='inner')
# Joining df with account_usage_df on account_id
DF = pd.merge(DF, ACC_USAGE_DF, on='account_id', how='inner')
# Joining df with churn_status_df on customer_id
DF = pd.merge(DF, CHURN_STATUS_DF, on='customer_id', how='inner')
# Joining df with city_df on area_id
DF = pd.merge(DF, CITY_DF, on='zip_code', how='inner')

In [5]:
DF.head()
DF.keys()

Index(['customer_id', 'gender', 'age', 'senior_citizen', 'married',
       'num_dependents', 'zip_code', 'account_id', 'tenure_months',
       'num_referrals', 'has_internet_service', 'internet_type',
       'has_unlimited_data', 'has_phone_service', 'has_multiple_lines',
       'has_premium_tech_support', 'has_online_security', 'has_online_backup',
       'has_device_protection', 'contract_type', 'paperless_billing',
       'payment_method', 'avg_long_distance_fee_monthly',
       'total_long_distance_fee', 'avg_gb_download_monthly', 'stream_tv',
       'stream_movie', 'stream_music', 'total_monthly_fee',
       'total_charges_quarter', 'total_refunds', 'status', 'churn_label',
       'churn_category', 'churn_reason', 'area_id', 'city', 'latitutde',
       'longitude', 'population'],
      dtype='object')

In [8]:
DF.head(1)
# Assuming you have a DataFrame called df
first_row_json = DF.head(1).to_json(orient='records')

print(first_row_json)

[{"customer_id":"0002-ORFBO","gender":"Female","age":37,"senior_citizen":"No","married":"Yes","num_dependents":0,"zip_code":93225,"account_id":"XSWV-PAYXZ","tenure_months":9,"num_referrals":2,"has_internet_service":"Yes","internet_type":"Cable","has_unlimited_data":"Yes","has_phone_service":"Yes","has_multiple_lines":"No","has_premium_tech_support":"Yes","has_online_security":"No","has_online_backup":"Yes","has_device_protection":"No","contract_type":"One Year","paperless_billing":"Yes","payment_method":"Credit Card","avg_long_distance_fee_monthly":42.39,"total_long_distance_fee":381.51,"avg_gb_download_monthly":16,"stream_tv":"Yes","stream_movie":"No","stream_music":"No","total_monthly_fee":65.6,"total_charges_quarter":593.3,"total_refunds":0.0,"status":"Stayed","churn_label":"No","churn_category":null,"churn_reason":null,"area_id":649,"city":"Frazier Park","latitutde":34.827662,"longitude":-118.999073,"population":4498}]


In [139]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
def convert_to_categorical(df:pd.DataFrame)->pd.DataFrame:
    """
    Converts the following columns to categorical data
    
    Yes No to 1/0
    - has_internet_service
    - has_phone_service
    - has_unlimited_data
    - has_multiple_lines
    - has_premium_tech_support
    - has_online_security
    - has_online_backup
    - has_device_protection
    - paperless_billing
    - stream_movie
    - stream_music
    - stream_tv
    - churn_label
    - senior_citizen
    - married
    - gender (1 for male, 0 for female)

    One-hot encodes the following columns
    - payment_method
    - internet_type

    Ordinal encodes the following columns
    - contract_type (one-year, month-to-month, two-year) => (1,0,2) 

    Label Encodes the following columns
    - status
    
    Binary encodes
    - churn_category

    """
    # Creating output_df
    output_df = df.copy()

    # Copying over account_id, customer_id
    output_df['account_id'] = df['account_id']
    output_df['customer_id'] = df['customer_id']
    output_df['zip_code'] = df['zip_code']

     # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0, "Male":1, "Female":0}
    output_df['has_internet_service'] = df['has_internet_service'].map(mapping)
    output_df['has_phone_service'] = df['has_phone_service'].map(mapping)
    output_df['has_unlimited_data'] = df['has_unlimited_data'].map(mapping)
    output_df['has_multiple_lines'] = df['has_multiple_lines'].map(mapping)
    output_df['has_premium_tech_support'] = df['has_premium_tech_support'].map(mapping)
    output_df['has_online_security'] = df['has_online_security'].map(mapping)
    output_df['has_online_backup'] = df['has_online_backup'].map(mapping)
    output_df['has_device_protection'] = df['has_device_protection'].map(mapping)
    output_df['paperless_billing'] = df['paperless_billing'].map(mapping)
    output_df['stream_movie'] = df['stream_movie'].map(mapping)
    output_df['stream_music'] = df['stream_music'].map(mapping)
    output_df['stream_tv'] = df['stream_tv'].map(mapping)
    output_df['senior_citizen'] = df['senior_citizen'].map(mapping)
    output_df['married'] = df['married'].map(mapping)
    output_df['gender'] = df['gender'].map(mapping)

    # Some are missing churn_labels, but have status as "churned"
    output_df['churn_label'] = df['churn_label']
    output_df.loc[df['status'] == 'Churned', 'churn_label'] = "Yes"
    output_df['churn_label'] = output_df['churn_label'].map(mapping)
    # output_df['churn_label'] = output_df['churn_label'].fillna(0)

    # Ordinal Encoding
    mapping = {'Month-to-Month':0, 'One Year':1, 'Two Year':2}
    output_df['contract_type'] = df['contract_type'].map(mapping)

    # One-hot encoding
    one_hot_encoder = OneHotEncoder()
    one_hot_encoded_cols = ["payment_method", "internet_type"]
    encoded_features = one_hot_encoder.fit_transform(df[one_hot_encoded_cols])
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=one_hot_encoder.get_feature_names_out(one_hot_encoded_cols))
    output_df = pd.concat([output_df, encoded_df], axis=1)

    # Label Encoding
    label_encoder = LabelEncoder()
    output_df['status'] = label_encoder.fit_transform(df['status'])

    # Binary Encoding
    binary_encoder = BinaryEncoder(cols=['churn_category'])
    binary_encoder.fit_transform(df['churn_category'])
    churn_cat = binary_encoder.transform(df['churn_category'])
    output_df = pd.concat([output_df, churn_cat], axis=1)

    # Copying over the numerical values
    output_df['num_referrals'] = df['num_referrals']
    output_df['tenure_months'] = df['tenure_months']
    output_df['avg_long_distance_fee_monthly'] = df[['avg_long_distance_fee_monthly']]
    output_df['total_long_distance_fee'] = df[['total_long_distance_fee']]
    output_df['avg_gb_download_monthly'] = df[['avg_gb_download_monthly']]
    output_df['total_monthly_fee'] = df[['total_monthly_fee']]
    output_df['total_charges_quarter'] = df[['total_charges_quarter']]
    output_df['total_refunds'] = df[['total_refunds']]
    output_df['population'] = df[['population']]

    # Dropping tables
    output_df = output_df.drop(columns=['internet_type', 'payment_method', 'churn_category', 'churn_reason'])

    return output_df


In [140]:
def scale_numerical_data(df: pd.DataFrame) -> Tuple[pd.DataFrame, MinMaxScaler]:
    """
    Scales all numerical data
    """
    scaler = MinMaxScaler()
    numerical_cols = ['num_referrals', 'age', 'tenure_months', 'avg_long_distance_fee_monthly', 'total_long_distance_fee', 'avg_gb_download_monthly', 'total_monthly_fee', 'total_charges_quarter', 'total_refunds', 'population']
    df.loc[:, numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df, scaler

In [129]:
from sklearn.decomposition import PCA

def perform_pca(df: pd.DataFrame, n_components: int):
    # Separate the features from the target variable (if applicable)
    features = df.drop(columns=['churn_label', 'status', 'customer_id', 'account_id', 'zip_code', 'city'])  # Replace 'target' with your churn_label variable column name

    # Perform PCA
    pca = PCA(n_components=n_components)
    transformed_features = pca.fit_transform(features)

    # Create a new DataFrame with the transformed features
    transformed_df = pd.DataFrame(data=transformed_features, columns=[f"PC{i+1}" for i in range(n_components)])

    # Concatenate the transformed features with the churn_label variable (if applicable)
    if 'churn_label' in df.columns:
        transformed_df['churn_label'] = df['churn_label']

    return transformed_df, pca


In [147]:
# Transforming DF
parsed_df = convert_to_categorical(DF)
parsed_df, scaler = scale_numerical_data(parsed_df)

In [148]:
transformed_df, pca = perform_pca(parsed_df, 5)
transformed_df

,PC1,PC2,PC3,PC4,PC5,churn_label
0,-118.052726,0.691531,0.090287,0.159955,-0.551817,0
1,-118.052763,0.766318,-0.866742,-0.265651,0.058195,0
2,-118.052813,0.820702,-1.377473,0.542717,0.278057,0
3,-118.052726,0.737594,-0.494861,0.214424,-0.427950,0
4,-583.050469,-0.876171,-0.075605,-0.776590,-0.251477,0
...,...,...,...,...,...,...
7038,-158.052109,0.217740,0.109590,-1.088779,-0.241642,0
7039,723.964045,1.023746,-0.544855,0.821999,-0.841849,0
7040,723.963982,1.034464,-0.454879,-1.056443,0.211924,0
7041,723.964015,1.047806,-0.501376,-0.728685,-0.092571,1


# Building the Model
* Splitting Dataset

In [158]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

# Split the transformed_df into features (X) and target variable (y)
X = transformed_df.drop('churn_label', axis=1)
y = transformed_df['churn_label']

# Perform train-test split while maintaining the class imbalance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Check the class distribution in the training set
class_counts = y_train.value_counts()
class_imbalance_ratio = class_counts[1] / class_counts[0]
print("Class Imbalance Ratio in Training Set:", class_imbalance_ratio)


# Create an instance of RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy=0.5, random_state=42)

# Resample the training data
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Check the class distribution after undersampling
class_counts_resampled = pd.Series(y_train_resampled).value_counts()
class_imbalance_ratio_resampled = class_counts_resampled[1] / class_counts_resampled[0]
print("Class Imbalance Ratio after Undersampling:", class_imbalance_ratio_resampled)


Class Imbalance Ratio in Training Set: 0.3611983570910848
Class Imbalance Ratio after Undersampling: 0.5


* Using XGBoost

In [159]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# Set up XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Set up StratifiedKFold with class imbalance
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store the evaluation results for each fold
fold_train_scores = []
fold_val_scores = []

# Iterate over the folds
# for train_index, val_index in kfold.split(X_train, y_train):
for train_index, val_index in kfold.split(X_train_resampled, y_train_resampled):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model
    xgb_model.fit(X_train_fold, y_train_fold)

    # Predict on the training and validation data
    y_train_pred = xgb_model.predict(X_train_fold)
    y_val_pred = xgb_model.predict(X_val_fold)

    # Calculate accuracy scores
    train_accuracy = accuracy_score(y_train_fold, y_train_pred)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)

    # Append scores to the lists
    fold_train_scores.append(train_accuracy)
    fold_val_scores.append(val_accuracy)

# Print the evaluation results for each fold
for i in range(len(fold_train_scores)):
    print(f"Fold {i+1}: Train Accuracy = {fold_train_scores[i]}, Validation Accuracy = {fold_val_scores[i]}")


Fold 1: Train Accuracy = 0.9986064659977704, Validation Accuracy = 0.8929765886287625
Fold 2: Train Accuracy = 0.9969342251950948, Validation Accuracy = 0.89520624303233
Fold 3: Train Accuracy = 0.9983277591973244, Validation Accuracy = 0.9141583054626533
Fold 4: Train Accuracy = 0.9980490523968785, Validation Accuracy = 0.8974358974358975
Fold 5: Train Accuracy = 0.9980490523968785, Validation Accuracy = 0.9018952062430323


### Evaluating on Test Set

In [161]:
# Predict on the test set
y_test_pred = xgb_model.predict(X_test)

# Calculate accuracy score on the test set
test_accuracy = accuracy_score(y_test, y_test_pred)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.8800567778566359


### Class Imbalance
Not churned = 5174 ~ 75%

Churned = 1817 ~25%

In [117]:
# Checking for class imbalance
print(parsed_df['churn_label'].value_counts())

churn_label
No    5174
1     1869
Name: count, dtype: int64


# Outlier Analysis

In [9]:
def detect_outliers_zscore(df:pd.DataFrame, threshold:int=3)->pd.DataFrame:
    """
    Detect outliers using the z-score method.
    Returns a boolean mask indicating the outliers.
    """
    z_scores = (df - np.mean(df)) / np.std(df)
    outliers = np.abs(z_scores) > threshold
    return outliers

def plot_boxplot(df_column:pd.Series)->None:
    """
    Plots a box plot based on the given DataFrame column using Plotly.
    """
    # Plotting boxplot using Plotly
    fig = px.box(df_column, title='Box Plot')

    # Show the plot
    fig.show()

def get_value_at_percentile(df:pd.DataFrame, column:str, percentile:int)->float:
    """
    Returns the value at a specific percentile in a DataFrame column.
    """
    # Get the values from the specified column
    column_values = df[column].values

    # Calculate the percentile value
    value_at_percentile = np.percentile(column_values, percentile)

    return value_at_percentile

def remove_rows_by_values(df:pd.DataFrame, column:str, lower_bound:float, upper_bound:float):
    """
    Removes rows from a DataFrame based on specific column values.
    """
    # Create a boolean mask to filter rows with the specified column values
    mask = (df[column] >= lower_bound) & (df[column] <= upper_bound)

    # Filter the DataFrame using the boolean mask
    updated_df = df[mask]

    return updated_df

# Outlier Analysis Summary
### ACC_DF

1. tenure_months

In the dataset, 362 rows have tenure_months at 72 while 0 rows have tenure_months at 71. As 72 months is 3 years, this suggests that the maximum contract period is 3 years. 362 rows consists of approximately 5% of the entire data set we have currently, hence I will not be removing any outliers based on tenure_months.

2. num_referrals

I decide to keep the 0th to 95th percentile of the rows based on the number of referrals. This meant the cutoff value would be 9, and 225 rows will be removed as a results.


In [ ]:
# ACC_DF = pd.read_csv(ACC_PATH)
# ACC_DF.head()
# # Keeping from 80th percentile to 0th percentile for tenure_months
# # plot_boxplot(ACC_DF['num_referrals'])
# get_value_at_percentile(ACC_DF, 'num_referrals', 80)

# def acc_df_remove_outliers(df:pd.DataFrame):
#     # Tenure_months: Keep from 0th to 80th percentile
#     # Num_referrals: Keep from 0th to 80th percentile
#     tenure_months_upper_bound = get_value_at_percentile(df, 'tenure_months', 90)
#     num_referarals_upper_bound = get_value_at_percentile(df, 'num_referrals', 80)

#     # Removing outliers
#     output_df = remove_rows_by_values(df, 'tenure_months', 0, tenure_months_upper_bound)
#     print(f"Removed {len(df) - len(output_df)} rows from the DataFrame based on the 'tenure_months' column values.")

#     output_df = remove_rows_by_values(df, 'num_referrals', 0, num_referarals_upper_bound)
#     print(f"Removed {len(df) - len(output_df)} rows from the DataFrame based on the 'num_referrals' column values.")

#     return output_df

# plot_boxplot(ACC_DF['tenure_months'])
# cleaned_acc_df = acc_df_remove_outliers(ACC_DF)
# plot_boxplot(cleaned_acc_df['tenure_months'])




### General Helper Functions

In [42]:
def get_unique_values(df:pd.DataFrame)->dict:
    """
    Runs through the columns of a dataframe and prints the unique values of each column. 
    """
    dict_unique_values = {}
    for cols in df.columns:
        dict_unique_values[cols] = df[cols].unique()
    return dict_unique_values

In [41]:
def get_NaN_count(df:pd.DataFrame)->dict:
    """
    Returns the number of NaN values for each column in a dictionary.
    """
    nan_count = df.isna().sum().to_dict()
    return nan_count

In [ ]:
def preprocess_acc_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocesses the account dataframe as below:

    Converts follow columns (yes/no) to 1/0
    - has_internet_service
    - has_phone_service
    - has_unlimited_data
    - has_multiple_lines
    - has_premium_tech_support
    - has_online_security
    - has_online_backup
    - has_device_protection
    - paperless_billing

    Converts follow columns to ordinal values:
    - contract_type (one-year, month-to-month, two-year) => (1,0,2) 

    One-hot encodes following columns:
    - payment_method
    - internet_type

    Scales following columns:
    - tenure_months
    """
    # Creating a new df
    output_df = pd.DataFrame()

    # Copying over account_id	customer_id	tenure_months
    output_df['account_id'] = df['account_id']
    output_df['customer_id'] = df['customer_id']

    # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0}

    output_df['has_internet_service'] = df['has_internet_service'].map(mapping)
    output_df['has_phone_service'] = df['has_phone_service'].map(mapping)
    output_df['has_unlimited_data'] = df['has_unlimited_data'].map(mapping)
    output_df['has_multiple_lines'] = df['has_multiple_lines'].map(mapping)
    output_df['has_premium_tech_support'] = df['has_premium_tech_support'].map(mapping)
    output_df['has_online_security'] = df['has_online_security'].map(mapping)
    output_df['has_online_backup'] = df['has_online_backup'].map(mapping)
    output_df['has_device_protection'] = df['has_device_protection'].map(mapping)
    output_df['paperless_billing'] = df['paperless_billing'].map(mapping)

    mapping = {'Month-to-Month':0, 'One Year':1, 'Two Year':2}
    output_df['contract_type'] = df['contract_type'].map(mapping)

    # One-hot encoding
    # one_hot_encoder = OneHotEncoder()
    # one_hot_encoded_cols = ["payment_method", "internet_type"]
    # encoded_df = pd.DataFrame(one_hot_encoder.fit_transform(df[one_hot_encoded_cols]))
    # encoded_df.columns = one_hot_encoder.get_feature_names_out(one_hot_encoded_cols)
    # print(encoded_df.head())

    # One-hot encoding
    one_hot_encoder = OneHotEncoder()
    one_hot_encoded_cols = ["payment_method", "internet_type"]
    encoded_features = one_hot_encoder.fit_transform(df[one_hot_encoded_cols])
    encoded_df = pd.DataFrame(encoded_features.toarray(), columns=one_hot_encoder.get_feature_names_out(one_hot_encoded_cols))
    output_df = pd.concat([output_df, encoded_df], axis=1)

    # Scaling
    scaler = MinMaxScaler()
    output_df['tenure_months'] = scaler.fit_transform(df[['tenure_months']])

    return output_df

### Account Usage 

In [ ]:
def preprocess_acc_usage_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the account usage dataframe as below:
    Scale the following columns:
    - avg_long_distance_fee_monthly
    - total_long_distance_fee
    - avg_gb_download_monthly
    - total_monthly_fee
    - total_chargers_quarter
    - total_refunds
    Converts following col to 1/0:
    - stream_move
    - stream_music
    - stream_tv
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over acc_id
    output_df['account_id'] = df['account_id']

    # Converting yes/no to 1/0
    mapping = {'Yes':1, 'No':0}

    output_df['stream_movie'] = df['stream_movie'].map(mapping)
    output_df['stream_music'] = df['stream_music'].map(mapping)
    output_df['stream_tv'] = df['stream_tv'].map(mapping)

    # Scaling
    scaler = MinMaxScaler()
    output_df['avg_long_distance_fee_monthly'] = scaler.fit_transform(df[['avg_long_distance_fee_monthly']])
    output_df['total_long_distance_fee'] = scaler.fit_transform(df[['total_long_distance_fee']])
    output_df['avg_gb_download_monthly'] = scaler.fit_transform(df[['avg_gb_download_monthly']])
    output_df['total_monthly_fee'] = scaler.fit_transform(df[['total_monthly_fee']])
    output_df['total_charges_quarter'] = scaler.fit_transform(df[['total_charges_quarter']])
    output_df['total_refunds'] = scaler.fit_transform(df[['total_refunds']])

    return output_df

### Churn Status

In [ ]:


def preprocess_churn_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the churn status dataframe as below:
    Convert following col to 1/0:
    - churn_label

    Binary encodes:
    - churn_category
    
    Label-encodes:
    - status

    Drops the following:
    - churn_reason -> Not planning to do NLP
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over acc_id
    output_df['customer_id'] = df['customer_id']

    # Converting yes/no to 1/0
    # Additional step to check status 
    output_df['churn_label'] = df['churn_label'].fillna(0)
    output_df.loc[df['status'] == 'Churned', 'churn_label'] = 1
    mapping = {'Yes':1, 'No':0, }
    output_df['churn_label'] = df['churn_label'].map(mapping)

    # Binary Encoding
    binary_encoder = BinaryEncoder(cols=['churn_category'])
    binary_encoder.fit_transform(df['churn_category'])
    churn_cat = binary_encoder.transform(df['churn_category'])
    output_df = pd.concat([output_df, churn_cat], axis=1)

    # Label Encoding
    label_encoder = LabelEncoder()
    output_df['status'] = label_encoder.fit_transform(df['status'])

    return output_df

### City DF

In [ ]:
def preprocess_city_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the city dataframe as below:
    Min-max scales population column
    Keep area_id and population only
    """
    # Creating new df
    output_df = pd.DataFrame()

    # Copying over area_id
    output_df['zip_code'] = df['zip_code']

    # Scaling
    scaler = MinMaxScaler()
    output_df['population'] = scaler.fit_transform(df[['population']])

    return output_df

### Customer DF

In [ ]:
def preprocess_customer_df(df:pd.DataFrame)->pd.DataFrame:
    """
    Preprocess the customer dataframe as below:
    Convert following col to 1/0:
    - senior_citizen
    - married
    - gender (1 for male, 0 for female)
    """
    # Create new df
    output_df = pd.DataFrame()

    # Copying over customer_id, zip_code
    output_df['customer_id'] = df['customer_id']
    output_df['zip_code'] = df['zip_code']

    # Converting columns to 1/0
    mapping = {'Yes':1, 'No':0, "Male":1, "Female":0}
    output_df['senior_citizen'] = df['senior_citizen'].map(mapping)
    output_df['married'] = df['married'].map(mapping)
    output_df['gender'] = df['gender'].map(mapping)

    return output_df

### Preprocessing all CSV

In [ ]:
# Loading dfs from CSV
ACCOUNT_DF = pd.read_csv(ACC_PATH)
ACCOUNT_USAGE_DF = pd.read_csv(ACC_USAGE_PATH)
CHURN_STATUS_DF = pd.read_csv(CHURN_STATUS_PATH)
CITY_DF = pd.read_csv(CITY_PATH)
CUSTOMER_DF = pd.read_csv(CUSTOMER_PATH)

# Preprocessing dfs
account_df = preprocess_acc_df(ACCOUNT_DF)
account_usage_df = preprocess_acc_usage_df(ACCOUNT_USAGE_DF)
churn_status_df = preprocess_churn_df(CHURN_STATUS_DF)
city_df = preprocess_city_df(CITY_DF)
customer_df = preprocess_customer_df(CUSTOMER_DF)

## Joining Tables

In [ ]:
# Joining customer and account tables on customer_id
df = pd.merge(customer_df, account_df, on='customer_id', how='inner')
# Joining df with account_usage_df on account_id
df = pd.merge(df, account_usage_df, on='account_id', how='inner')
# Joining df with churn_status_df on customer_id
df = pd.merge(df, churn_status_df, on='customer_id', how='inner')
# Joining df with city_df on area_id
df = pd.merge(df, city_df, on='zip_code', how='inner')